In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

import pandas as pd

import random
import pickle

import cv2

## Load Data from the Dataset and store them in X, y lists

In [ ]:
train_data = []
with open('german-traffic-signs/train.p', 'rb') as f:
    train_data = pickle.load(f)
    
valid_data = []
with open('german-traffic-signs/valid.p', 'rb') as f:
    valid_data = pickle.load(f)

test_data = []
with open('german-traffic-signs/test.p', 'rb') as f:
    test_data = pickle.load(f)

In [ ]:
X_train, y_train = train_data['features'], train_data['labels']
X_valid, y_valid = valid_data['features'], valid_data['labels']
X_test, y_test = test_data['features'], test_data['labels']

In [ ]:
df = pd.read_csv('german-traffic-signs/signnames.csv')

## Distribution of the Dataset 

In [ ]:
num_of_samples = []
 
cols = 5
num_classes = 43

fig, axs = plt.subplots(nrows=num_classes, ncols = cols, figsize=(5, 60))
fig.tight_layout()
for i in range(cols):
    for j,row in df.iterrows():
        x_selected = X_train[y_train == j]
        axs[j][i].imshow(x_selected[random.randint(0, len(x_selected - 1)), :, :], cmap=plt.get_cmap("gray"))
        axs[j][i].axis("off")
        if i == 2:
            axs[j][i].set_title(str(j) + '-' + row['SignName'])
            num_of_samples.append(len(x_selected))


plt.figure(figsize=(12, 4))
plt.bar(range(0, num_classes), num_of_samples)
plt.title("Distribution of the training dataset")
plt.xlabel("Class number")
plt.ylabel("Number of images")

In [ ]:
plt.imshow(X_train[1000])
plt.axis('off')
print(df['SignName'][y_train[1000]])

## Functions for image preprocessing

In [ ]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
img = grayscale(X_train[1000])
plt.imshow(img, cmap=plt.get_cmap("gray"))

In [ ]:
## Histogram Equalization - Normalize brightness, lead to high contrast which helps feature extraction
## Standardize brightness across pixels.
def hist_equalize(img):
    return cv2.equalizeHist(img)
    

In [ ]:
nimg = hist_equalize(img)
plt.imshow(nimg, cmap=plt.get_cmap('gray'))
plt.axis('off')

In [ ]:
def preprocessing(img):
    img = grayscale(img)
    img = hist_equalize(img)
    img = img/255
    return img

In [ ]:
X_train = np.array(list(map(preprocessing, X_train)))
X_valid = np.array(list(map(preprocessing, X_valid)))
X_test = np.array(list(map(preprocessing, X_test)))



In [ ]:
plt.imshow(X_train[1000], cmap=plt.get_cmap('gray'))

In [ ]:
X_train = X_train.reshape(len(X_train), 32, 32, 1)
X_valid = X_valid.reshape(len(X_valid), 32, 32, 1)
X_test = X_test.reshape(len(X_test), 32, 32, 1)

## Generate augmented images 

In [ ]:
datagen = ImageDataGenerator(width_shift_range=0.1,
                          height_shift_range=0.1,
                          zoom_range=0.2,
                          shear_range=0.1,
                          rotation_range=10)

datagen.fit(X_train)

## Convert labels to categorical and create/train the model for classification

In [ ]:
y_train = to_categorical(y_train, 43)
y_valid = to_categorical(y_valid, 43)
y_test = to_categorical(y_test, 43)

In [ ]:
def leNet_model(no_classes):
    model = Sequential()
    model.add(Conv2D(filters=60, kernel_size=(5,5), input_shape=(32,32,1), activation='relu'))
    model.add(Conv2D(filters=60, kernel_size=(5,5), input_shape=(32,32,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(filters=30, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(filters=30, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(rate=0.5))

    model.add(Flatten())
    model.add(Dense(units=500, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(units=no_classes, activation='softmax'))
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
model = leNet_model(43)

# h = model.fit(x=X_train, y=y_train, epochs=10, validation_data=(X_valid, y_valid), batch_size=400, verbose=1, shuffle=1)
h = model.fit_generator(datagen.flow(X_train, y_train, batch_size=50), steps_per_epoch=695, epochs=10, validation_data=(X_valid, y_valid), shuffle=1)

In [ ]:
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)